In [1]:
# Part a)
# define Least unit cost rule function
def luc (demand, setupcost, holdingcost):
    "Least unit cost algorithm"
    
    import numpy as np
    
    periods = len(demand)
    lotsize = np.zeros(periods)

    k = 0 #current period
    z = k + 1 #period to compare with current period
    tcost = setupcost #total cost to order from period k UNTIL period z
                      #initial value = setup cost 
    tdemand = demand[k] #sum of demand from period k UNTIL period z
                        #initial value = demand in period k
    lotsize[k] = demand[k]
    
    totalcost = setupcost #total cost of the order strategy
    
    while z < periods:
        new_tcost = tcost + holdingcost * (z-k) * demand[z]
        new_tdemand = tdemand + demand[z]
        
        if (tcost / tdemand) >= (new_tcost / new_tdemand): #compare cost per unit
            #cost per unit NOT increase => include demand of period z in current lot size
            lotsize[k] += demand[z]
            totalcost += holdingcost * (z-k) * demand[z] #update total cost by holding cost of new future demand of period z
            
            tcost = new_tcost
            tdemand = new_tdemand
            z = z+1
        else: #cost per unit increase => start a new lotsize          
            k = z 
            z = k+1 
            tcost = setupcost
            tdemand = demand[k]
            
            lotsize[k] = demand[k]
            totalcost += setupcost #update total cost by setup cost for new order
    
    print("Lot-size orders in each period are: ")
    print(lotsize)
    print("Total ordering cost is ", totalcost)
    return (lotsize, totalcost);

In [2]:
# apply LUC function with data
demand = [750, 100, 50, 100, 400, 1000]
setupcost = 400
holdingcost = 2

(forecast, tc) =luc(demand, setupcost, holdingcost)

Lot-size orders in each period are: 
[ 750.  150.    0.  500.    0. 1000.]
Total ordering cost is  2500


In [3]:
# Part b)
# define EOQ function
def eoq_func (demand, setupcost, holdingcost, usage, lagrange):
    "Function returns EOQ and used capacity for multiple products"
    
    if not (len(demand) == len(setupcost) and len(setupcost) == len(holdingcost) and len(holdingcost) == len(usage)):
    # check whether all products have all required information    
        print("Not enough information to calculate!")
    else: 
        import math
        import numpy as np
        
        eoq = np.zeros(len(demand))
        used_capacity = 0
        for i in range(len(demand)):
            eoq[i] = math.sqrt(2 * demand[i] * setupcost[i] / (holdingcost[i] + 2 * usage[i] * lagrange))
            used_capacity += usage[i] * eoq[i]
        
        return (eoq, used_capacity);

    
# define Lagrange multiplier EOQ function for Warehouse scheduling problem
def lagrange_func (demand, setupcost, holdingcost, usage, capacity = None):
    "Lagrange multiplier solution for Warehouse scheduling problem"
    
    # Unconstrained case
    if capacity is None:
        lagrange = 0
        (eoq, used_capacity) = eoq_func(demand, setupcost, holdingcost, usage, lagrange)
    
    # Constrained case
    else:
        lb = 0
        ub = max(demand)  

        while (ub-lb > 0.01):
            lagrange = 0.5*(lb+ub)
            (eoq, used_capacity) = eoq_func(demand, setupcost, holdingcost, usage, lagrange)
            if used_capacity > capacity: 
                lb = lagrange
            else: ub = lagrange

    print("Unconstrained solution is:")
    print(eoq)
    print("Used capacity: ", used_capacity)
    print("Lagrange multiplier:", lagrange)
    return (eoq, used_capacity, lagrange);


In [4]:
# applied Langrage function with data
setupcost = [500, 1000, 700]
holdingcost = [3, 5, 10]
demand = [50, 200, 125]
usage = [1, 2, 1]
capacity = 500

(eoq, used_capacity, lagrange) = lagrange_func(demand, setupcost, holdingcost, usage, capacity)

Unconstrained solution is:
[ 76.78255698 158.3024045  106.32126171]
Used capacity:  499.70862769151796
Lagrange multiplier: 2.740478515625
